In [24]:
%matplotlib inline

In [25]:
import numpy as np
import pandas as pd
from sparsesvd import sparsesvd 
from scipy.sparse import csc_matrix

In [26]:
data = pd.read_csv('data/data.txt', sep=' ', header = None)
X = np.array(data.T)

In [4]:
%%file SSVD.cpp
#include <iostream>
#include <fstream>
#include <Eigen/Dense>
#include <functional>

using std::cout;
using std::endl;
using std::ofstream;
using std::sqrt;
using std::abs;
using std::pow;
using std::log;
using std::sort;
using std::unique;

        
int main() 
{
    using namespace Eigen;
    
    MatrixXd X(3, 4);
    X << 5.17237, 3.73572, 6.29422, 6.55268,
    5.33713, 3.88883, 1.93637, 4.39812,
    8.22086, 6.94502, 6.36617,  6.5961;
    
    double gamma1 = 2;
    double gamma2 = 2;
    double tol = 1e-4;
    int max_iter = 10;
    double inf = std::numeric_limits<double>::infinity();
    
    JacobiSVD<MatrixXd> svd(X, ComputeThinU | ComputeThinV);
    VectorXd s = svd.singularValues().head(1);
    MatrixXd u = svd.matrixU().leftCols(1);
    MatrixXd v = svd.matrixV().leftCols(1);
    
    int n = X.rows();
    int d = X.cols();
    double u_delta = 1;
    double v_delta = 1;
    int niter = 0;
    double SST = X.unaryExpr([](double x) { return pow(x, 2); }).sum();
    
    MatrixXd Xt, Xu, w2_inv, v_temp, v_temp_t, v_new;
    VectorXd lambda2s(d+1);
    double sigma_sq2, lambda2_temp, lambda2, part3, part4;
    VectorXd BIC2s, sign, part1, part2, best2_sign, best2_part1, best2_part2;    
    
    MatrixXd Xv, w1_inv, u_temp, v_t, u_new;
    VectorXd lambda1s(n+1);
    double sigma_sq1, lambda1_temp, lambda1, part7, part8;
    VectorXd BIC1s, sign1, part5, part6, best1_sign1, best1_part5, best1_part6;

    
    while((u_delta > tol) | (v_delta > tol)){
        niter += 1;
    
        // update v
        Xt = X.transpose();
        Xu = Xt * u;
        w2_inv = pow(abs(Xu.array()), gamma2);

        sigma_sq2 = abs(SST - pow(Xu.array(), 2).sum())/(double)(n*d-d);


        lambda2s << abs(Xu.array() * w2_inv.array()), 0.0;
        unique(lambda2s.data(), lambda2s.data()+lambda2s.size());
        sort(lambda2s.data(), lambda2s.data()+lambda2s.size());
        
        BIC2s = VectorXd::Ones(lambda2s.size() - 1) * inf;

        int best2 = 0;
        for(int i = 0; i < BIC2s.size(); i++){
            lambda2_temp = lambda2s[i];
            sign =  Xu.unaryExpr([](double x) { return (0.0 < x) - (x < 0.0); }).array();
            part1 = (abs(Xu.array()) >= (lambda2_temp / w2_inv.array())).array().cast<double>();
            part2 = (abs(Xu.array()) - (lambda2_temp / w2_inv.array())).array();

            v_temp = sign.array() * part1.array() * part2.array();
            v_temp_t = v_temp.transpose();

            part3 = pow((X - u * v_temp_t).array(), 2).sum()/sigma_sq2/(n*d);
            part4 = (v_temp.array() != 0).array().cast<double>().sum()*log(n*d)/(n*d);
            BIC2s[i] = part3 + part4;

            if(i > 0 && BIC2s[i] < BIC2s[i-1]){
                best2 = i;
            }
        }

        lambda2 = lambda2s[best2];
        best2_sign =  Xu.unaryExpr([](double x) { return (0.0 < x) - (x < 0.0); }).array();
        best2_part1 = (abs(Xu.array()) >= (lambda2 / w2_inv.array())).array().cast<double>();
        best2_part2 = (abs(Xu.array()) - (lambda2 / w2_inv.array())).array();

        v_new = best2_sign.array() * best2_part1.array() * best2_part2.array();
        v_new = v_new.array()/sqrt(pow(v_new.array(), 2).sum());

        v_delta = sqrt(pow((v.array()-v_new.array()), 2).sum());
        v = v_new;

        // update u
        Xv = X * v;
        w1_inv = pow(abs(Xv.array()), gamma1);

        sigma_sq1 = abs(SST - pow(Xv.array(), 2).sum())/(double)(n*d-n);

        lambda1s << abs(Xv.array() * w1_inv.array()), 0.0;
        unique(lambda1s.data(), lambda1s.data()+lambda1s.size());
        sort(lambda1s.data(), lambda1s.data()+lambda1s.size());

        BIC1s = VectorXd::Ones(lambda1s.size() - 1) * inf;

        int best1 = 0;
        for(int i = 0; i < BIC1s.size(); i++){
            double lambda1_temp = lambda1s[i];
            sign1 =  Xv.unaryExpr([](double x) { return (0.0 < x) - (x < 0.0); }).array();
            part5 = (abs(Xv.array()) >= (lambda1_temp / w1_inv.array())).array().cast<double>();
            part6 = (abs(Xv.array()) - (lambda1_temp / w1_inv.array())).array();

            u_temp = sign1.array() * part5.array() * part6.array();
            v_t = v.transpose();

            part7 = pow((X - u_temp * v_t).array(), 2).sum()/sigma_sq1/(n*d);
            part8 = (u_temp.array() != 0).array().cast<double>().sum()*log(n*d)/(n*d);
            BIC1s[i] = part7 + part8;

            if(i > 0 && BIC1s[i] < BIC1s[i-1]){
                best1 = i;
            }
        }

        lambda1 = lambda1s[best1];
        best1_sign1 =  Xv.unaryExpr([](double x) { return (0.0 < x) - (x < 0.0); }).array();
        best1_part5 = (abs(Xv.array()) >= (lambda1 / w1_inv.array())).array().cast<double>();
        best1_part6 = (abs(Xv.array()) - (lambda1 / w1_inv.array())).array();

        u_new = best1_sign1.array() * best1_part5.array() * best1_part6.array();
        u_new = u_new.array()/sqrt(pow(u_new.array(), 2).sum());

        u_delta = sqrt(pow((u.array()-u_new.array()), 2).sum());
        u = u_new;
    
        if(niter > max_iter){
            cout << "Fail to converge! Increase the max_iter!" << "\n\n";
            break;
        }
    }

    cout << niter;

}

Overwriting SSVD.cpp


In [5]:
%%bash

g++ -o SSVD.exe SSVD.cpp -std=c++14 -I/usr/include/eigen3

In [6]:
%%bash

./SSVD.exe

1

In [7]:
%%file wrap.cpp
<%
cfg['compiler_args'] = ['-std=c++11']
cfg['include_dirs'] = ['../notebooks/eigen3']
setup_pybind11(cfg)
%>

#include <pybind11/pybind11.h>
#include <pybind11/eigen.h>
#include <Eigen/Dense>

namespace py = pybind11;
        
int SSVD(Eigen::MatrixXd X){
    
    double gamma1 = 2;
    double gamma2 = 2;
    double tol = 1e-4;
    int max_iter = 10;
    double inf = std::numeric_limits<double>::infinity();
    
    Eigen::JacobiSVD<Eigen::MatrixXd> svd(X, Eigen::ComputeThinU | Eigen::ComputeThinV);
    Eigen::VectorXd s = svd.singularValues().head(1);
    Eigen::MatrixXd u = svd.matrixU().leftCols(1);
    Eigen::MatrixXd v = svd.matrixV().leftCols(1);
    
    int n = X.rows();
    int d = X.cols();
    double u_delta = 1;
    double v_delta = 1;
    int niter = 0;
    double SST = X.unaryExpr([](double x) { return pow(x, 2); }).sum();
    
    Eigen::MatrixXd Xt, Xu, w2_inv, v_temp, v_temp_t, v_new;
    Eigen::VectorXd lambda2s(d+1);
    double sigma_sq2, lambda2_temp, part3, part4;
    Eigen::VectorXd BIC2s, sign, part1, part2;    
    
    Eigen::MatrixXd Xv, w1_inv, u_temp, v_t, u_new;
    Eigen::VectorXd lambda1s(n+1);
    double sigma_sq1, lambda1_temp, part7, part8;
    Eigen::VectorXd BIC1s, sign1, part5, part6;
    
    double best1_BIC, best2_BIC;
    bool converge = true;
    
    while((u_delta > tol) | (v_delta > tol)){
        niter += 1;
    
        // update v
        Xt = X.transpose();
        Xu = Xt * u;
        w2_inv = pow(abs(Xu.array()), gamma2);

        sigma_sq2 = abs(SST - pow(Xu.array(), 2).sum())/(double)(n*d-d);


        lambda2s << abs(Xu.array() * w2_inv.array()), 0.0;
        std::unique(lambda2s.data(), lambda2s.data()+lambda2s.size());
        std::sort(lambda2s.data(), lambda2s.data()+lambda2s.size());

        BIC2s = Eigen::VectorXd::Ones(lambda2s.size() - 1) * inf;

        best2_BIC = inf;
        sign =  Xu.unaryExpr([](double x) { return (0.0 < x) - (x < 0.0); }).array().cast<double>();
        for(int i = 0; i < BIC2s.size(); i++){
            lambda2_temp = lambda2s[i];
            part1 = (abs(Xu.array()) >= (lambda2_temp / w2_inv.array())).array().cast<double>();
            part2 = (abs(Xu.array()) - (lambda2_temp / w2_inv.array())).array();

            v_temp = sign.array() * part1.array() * part2.array();
            v_temp_t = v_temp.transpose();

            part3 = pow((X - u * v_temp_t).array(), 2).sum()/sigma_sq2/(n*d);
            part4 = (v_temp.array() != 0).array().cast<double>().sum()*log(n*d)/(n*d);
            BIC2s[i] = part3 + part4;

            if(BIC2s[i] < best2_BIC){
                best2_BIC = BIC2s[i];
                v_new = v_temp;
            }  
        }

        v_new = v_new.array()/sqrt(pow(v_new.array(), 2).sum());

        v_delta = sqrt(pow((v.array()-v_new.array()), 2).sum());
        v = v_new;

        
        // update u
        Xv = X * v;
        w1_inv = pow(abs(Xv.array()), gamma1);

        sigma_sq1 = abs(SST - pow(Xv.array(), 2).sum())/(double)(n*d-n);

        lambda1s << abs(Xv.array() * w1_inv.array()), 0.0;
        std::unique(lambda1s.data(), lambda1s.data()+lambda1s.size());
        std::sort(lambda1s.data(), lambda1s.data()+lambda1s.size());

        BIC1s = Eigen::VectorXd::Ones(lambda1s.size() - 1) * inf;

        best1_BIC = inf;
        sign1 =  Xv.unaryExpr([](double x) { return (0.0 < x) - (x < 0.0); }).array().cast<double>();
        for(int i = 0; i < BIC1s.size(); i++){
            lambda1_temp = lambda1s[i];
            part5 = (abs(Xv.array()) >= (lambda1_temp / w1_inv.array())).array().cast<double>();
            part6 = (abs(Xv.array()) - (lambda1_temp / w1_inv.array())).array();

            u_temp = sign1.array() * part5.array() * part6.array();
            v_t = v.transpose();

            part7 = pow((X - u_temp * v_t).array(), 2).sum()/sigma_sq1/(n*d);
            part8 = (u_temp.array() != 0).array().cast<double>().sum()*log(n*d)/(n*d);
            BIC1s[i] = part7 + part8;

            if(BIC1s[i] < best1_BIC){
                best1_BIC = BIC1s[i];
                u_new = u_temp;
            }
        }

        u_new = u_new.array()/sqrt(pow(u_new.array(), 2).sum());

        u_delta = sqrt(pow((u.array()-u_new.array()), 2).sum());
        u = u_new;

        
        if(niter > max_iter){
            converge = false;
            break;
        }
    }
    
    return niter;
}

PYBIND11_PLUGIN(wrap) {
    pybind11::module m("wrap", "auto-compiled c++ extension");
    m.def("SSVD", &SSVD);
    return m.ptr();
}

Writing wrap.cpp


In [4]:
%%file wrap.cpp
<%
cfg['compiler_args'] = ['-std=c++11']
cfg['include_dirs'] = ['../notebooks/eigen3']
setup_pybind11(cfg)
%>

#include <pybind11/pybind11.h>
#include <pybind11/eigen.h>
#include <Eigen/Dense>

namespace py = pybind11;

struct Decomposition {
    Decomposition(const Eigen::MatrixXd X) : X(X) { }
    void SSVD() { 
 
        double gamma1 = 2;
        double gamma2 = 2;
        double tol = 1e-4;
        int max_iter = 10;
        double inf = std::numeric_limits<double>::infinity();

        Eigen::JacobiSVD<Eigen::MatrixXd> svd(X, Eigen::ComputeThinU | Eigen::ComputeThinV);
        Eigen::VectorXd s = svd.singularValues().head(1);
        Eigen::MatrixXd u = svd.matrixU().leftCols(1);
        Eigen::MatrixXd v = svd.matrixV().leftCols(1);

        int n = X.rows();
        int d = X.cols();
        double u_delta = 1;
        double v_delta = 1;
        int niter = 0;
        double SST = X.unaryExpr([](double x) { return pow(x, 2); }).sum();

        Eigen::MatrixXd Xt, Xu, w2_inv, v_temp, v_temp_t, v_new;
        Eigen::VectorXd lambda2s(d+1);
        double sigma_sq2, lambda2_temp, part3, part4;
        Eigen::VectorXd BIC2s, sign, part1, part2;    

        Eigen::MatrixXd Xv, w1_inv, u_temp, v_t, u_new;
        Eigen::VectorXd lambda1s(n+1);
        double sigma_sq1, lambda1_temp, part7, part8;
        Eigen::VectorXd BIC1s, sign1, part5, part6;

        double best1_BIC, best2_BIC;
        bool converge = true;

        while((u_delta > tol) | (v_delta > tol)){
            niter += 1;

            // update v
            Xt = X.transpose();
            Xu = Xt * u;
            w2_inv = pow(abs(Xu.array()), gamma2);

            sigma_sq2 = abs(SST - pow(Xu.array(), 2).sum())/(double)(n*d-d);


            lambda2s << abs(Xu.array() * w2_inv.array()), 0.0;
            std::unique(lambda2s.data(), lambda2s.data()+lambda2s.size());
            std::sort(lambda2s.data(), lambda2s.data()+lambda2s.size());

            BIC2s = Eigen::VectorXd::Ones(lambda2s.size() - 1) * inf;

            best2_BIC = inf;
            sign =  Xu.unaryExpr([](double x) { return (0.0 < x) - (x < 0.0); }).array().cast<double>();
            for(int i = 0; i < BIC2s.size(); i++){
                lambda2_temp = lambda2s[i];
                part1 = (abs(Xu.array()) >= (lambda2_temp / w2_inv.array())).array().cast<double>();
                part2 = (abs(Xu.array()) - (lambda2_temp / w2_inv.array())).array();

                v_temp = sign.array() * part1.array() * part2.array();
                v_temp_t = v_temp.transpose();

                part3 = pow((X - u * v_temp_t).array(), 2).sum()/sigma_sq2/(n*d);
                part4 = (v_temp.array() != 0).array().cast<double>().sum()*log(n*d)/(n*d);
                BIC2s[i] = part3 + part4;

                if(BIC2s[i] < best2_BIC){
                    best2_BIC = BIC2s[i];
                    v_new = v_temp;
                }  
            }

            v_new = v_new.array()/sqrt(pow(v_new.array(), 2).sum());

            v_delta = sqrt(pow((v.array()-v_new.array()), 2).sum());
            v = v_new;


            // update u
            Xv = X * v;
            w1_inv = pow(abs(Xv.array()), gamma1);

            sigma_sq1 = abs(SST - pow(Xv.array(), 2).sum())/(double)(n*d-n);

            lambda1s << abs(Xv.array() * w1_inv.array()), 0.0;
            std::unique(lambda1s.data(), lambda1s.data()+lambda1s.size());
            std::sort(lambda1s.data(), lambda1s.data()+lambda1s.size());

            BIC1s = Eigen::VectorXd::Ones(lambda1s.size() - 1) * inf;

            best1_BIC = inf;
            sign1 =  Xv.unaryExpr([](double x) { return (0.0 < x) - (x < 0.0); }).array().cast<double>();
            for(int i = 0; i < BIC1s.size(); i++){
                lambda1_temp = lambda1s[i];
                part5 = (abs(Xv.array()) >= (lambda1_temp / w1_inv.array())).array().cast<double>();
                part6 = (abs(Xv.array()) - (lambda1_temp / w1_inv.array())).array();

                u_temp = sign1.array() * part5.array() * part6.array();
                v_t = v.transpose();

                part7 = pow((X - u_temp * v_t).array(), 2).sum()/sigma_sq1/(n*d);
                part8 = (u_temp.array() != 0).array().cast<double>().sum()*log(n*d)/(n*d);
                BIC1s[i] = part7 + part8;

                if(BIC1s[i] < best1_BIC){
                    best1_BIC = BIC1s[i];
                    u_new = u_temp;
                }
            }

            u_new = u_new.array()/sqrt(pow(u_new.array(), 2).sum());

            u_delta = sqrt(pow((u.array()-u_new.array()), 2).sum());
            u = u_new;


            if(niter > max_iter){
                converge = false;
                break;
            }
        }

        niter_result = niter;
        u_result = u;
        v_result = v;
        s_result = s;
        converge_result = converge;
    }
    const Eigen::MatrixXd getX() const { return X; }
    const Eigen::MatrixXd getU() const { return u_result; }
    const Eigen::MatrixXd getV() const { return v_result; }
    const Eigen::VectorXd getS() const { return s_result; }
    const int getNiter() const { return niter_result; }
    const bool getConverge() const { return converge_result; }

    Eigen::MatrixXd X;
    Eigen::MatrixXd u_result;
    Eigen::MatrixXd v_result;
    Eigen::VectorXd s_result;
    int niter_result;
    bool converge_result;
 };


PYBIND11_MODULE(wrap, m) {
     py::class_<Decomposition>(m, "Decomposition")
         .def(py::init<const Eigen::MatrixXd>())
         .def("SSVD", &Decomposition::SSVD)
         .def("getX", &Decomposition::getX)
         .def("getU", &Decomposition::getU)
         .def("getV", &Decomposition::getV)
         .def("getS", &Decomposition::getS)
         .def("getNiter", &Decomposition::getNiter)
         .def("getConverge", &Decomposition::getConverge);

}

Writing wrap.cpp


In [5]:
import cppimport
import numpy as np
cppimport.force_rebuild()
code = cppimport.imp("wrap")

In [9]:
X = np.array([
    [5.17237, 3.73572, 6.29422, 6.55268],
    [5.33713, 3.88883, 1.93637, 4.39812],
    [8.22086, 6.94502, 6.36617,  6.5961]
])

In [10]:
code.SSVD(X)

1

In [12]:
import numpy as np
np.random.seed(54321)
u_tilde = np.r_[np.arange(3,11)[::-1], 2*np.ones(17), np.zeros(75)].reshape((-1,1))
u = u_tilde/np.linalg.norm(u_tilde)
v_tilde = np.r_[np.array([10,-10,8,-8,5,-5]),3*np.ones(5),-3*np.ones(5),np.zeros(34)].reshape((-1,1))
v = v_tilde/np.linalg.norm(v_tilde)
s = 50

X_star = s*u@v.T
n, d = X_star.shape
X_sim = X_star + np.random.randn(n,d)

In [27]:
%%time
p = code.Decomposition(X)
p.SSVD()

CPU times: user 11min 1s, sys: 4.29 s, total: 11min 6s
Wall time: 11min 15s


In [23]:
p.getNiter()

4